# Computational Methods in Economics (winter term 2019/20)

## Problem Set 3

#### DEADLINE: Wednesday, December 18, 12 pm (Noon)

### Preliminaries

#### Import Modules

In [1]:
import numpy as np
import scipy.linalg
import scipy.optimize

import matplotlib.pyplot as plt
%matplotlib inline

## Question 1

Write a function **mybisect(f, a, b)** in Python that implements the pseudo-code for the bisection method from the lecture, with initial interval **[a,b]**. DO NOT USE SciPy's in-built function! 

*Hint*: Consider using the **abs()** and **np.sign()** functions.  

In [2]:
def mybisect(fun, a, b):
    """
    Implements the bisection method
    
    (fun, float, float) -> float
    """
    ### BEGIN SOLUTION
    ## check on signs of function values at initial points a and b
    assert np.sign(fun(a)) == - np.sign(fun(b)), "Choose a and b such that f(a) and f(b) have different signs"
    
    # iteration counter
    it = 0
    # choose tolerance level
    tol = 1e-10
    # initialize d (= function value at x)
    d = 1
    # while-loop: iterate until d sufficiently small
    while abs(d) > tol:
        it += 1
        # find intermediate value between a and b
        x = (a + b)/2
        # evaluate function
        d = fun(x)
        # find new end points for interval [a,b]
        if np.sign(d) == np.sign(fun(a)):
            a = x
        elif np.sign(d) == np.sign(fun(b)):
            b = x
    
#     print("Number of iterations = {}".format(it) )
    
    return x
    ### END SOLUTION

In [3]:
# test function
def fun(x):
    return np.sin(4 * (x - 0.25)) + x + x**20 - 1

assert np.allclose(mybisect(fun, 0, 2), 0.408293504267931)

In [4]:
# THIS IS A TEST CELL!
### BEGIN HIDDEN TESTS
assert np.allclose( mybisect(lambda x: -x**2 + 4, 1, 10), 2)
assert np.allclose( mybisect(lambda x: -x**2 + 4, -1, -10), -2)
### END HIDDEN TESTS

## Question 2

In this question, we use Newton's method to solve a Cournot duopoly. There two firms producing $q_i$ units of a good, for $i = 1,2$. Their production cost are given by:

$$
C_i(q_i) = 0.5c_i q_i^2.
$$

The inverse demand in the market is given by 

$$
P(q) = q^{-1/\eta} = (q_1 + q_2)^{-1/\eta}.
$$

Hence, the profit of firm $i$ depends on the quantity produced by the other firm, and vice versa:

$$
  \pi_i(q_1, q_2) = P(q_1 + q_2)q_i - C_i(q_i)
$$  

Taking first-order conditions gives the following equilibrium condition for firm $i$:

$$
   \partial \pi_i / \partial q_i = 0 = P(q_1 + q_2) + P'(q_1 + q_2) q_i - C_i'(q_i) \equiv \phi_i(q_1, q_2)
$$    

Hence, the equilibrium can be characterized by a system of non-linear equations: 

$$
    \phi(q_1, q_2) = \begin{bmatrix} \phi_1(q_1, q_2) \\
                                     \phi_2(q_1, q_2) \end{bmatrix} = 
                     \begin{bmatrix} 0 \\
                                     0 \end{bmatrix}
$$                                     
                                     

(a) Write a function **cournot** that implements the system of non-linear equations above. As inputs, it takes an array **q = [q_1, q_2]**, an array **c = [c_1, c_2]**, and a scalar **eta** ($=\eta$).

(b) Write a function **cournot_J** that takes the same inputs as **cournot** and implements the Jacobian of the system.

(c) Write a function **my_newton** that takes an array **q** (initial guess), two functions **cournot** and **cournot_J**, as well as **c** and **eta**, as inputs and solves for the Cournot equilibrium. Make sure that your algorithm does not run indefinitely by including an upper bound for the number of iterations (**maxit**).

In [5]:
## (a)

def cournot(q, c, eta):
    """
    Implements the Cournot duopoly as a system of non-linear equations in two unknowns
   
    ((2,) np.array, (2,) np.array, float) -> (2,) np.array   
    """
    ## BEGIN SOLUTION 
    # phi_i(q1, q2) = [(q1 + q2)^(-1/eta) - (1/eta)*[(q1 + q2)^(-1/eta - 1)]*q_i - c_i*q_i]
    e = -1/eta
    
    return np.array(( (q[0]+q[1])**e + e*((q[0]+q[1])**(e-1))*q[0] - c[0]*q[0],
                    (q[0]+q[1])**e + e*((q[0]+q[1])**(e-1))*q[1] - c[1]*q[1]))
    ## END SOLUTION

In [6]:
assert np.allclose( cournot([2, 2], [0.5, 0.5], 1.5), np.array([-0.73543316, -0.73543316]))

In [7]:
# THIS IS A TEST CELL!
### BEGIN HIDDEN TESTS
assert np.allclose( cournot([0.8395676,  0.68879643], [0.6, 0.8], 1.6), np.zeros(2))
### END HIDDEN TESTS

In [8]:
## (b)

def cournot_J(q, c, eta):
    """
    Implements the Jacobian of the Cournot duopoly model 
    
    ((2, 2) np.array, (2, 2) np.array, float) -> (2, 2) np.array  
    """
    ### BEGIN SOLUTION
    e = -1/eta
    
    f_00 = (e*(q[0]+q[1])**(e-2))*(e*q[0] + q[0] + 2*q[1]) - c[0] 
    f_01 = (e*(q[0]+q[1])**(e-2))*(e*q[0] + q[1])
    f_10 = (e*(q[0]+q[1])**(e-2))*(e*q[1] + q[0])
    f_11 = (e*(q[0]+q[1])**(e-2))*(e*q[1] + 2*q[0] + q[1]) - c[1]
    
    return np.array([[f_00, f_01], [f_10, f_11]])
    ### END SOLUTION

In [9]:
assert np.allclose(cournot_J([2, 2], [0.5, 0.5], 1.5), np.array([[-0.57716533, -0.01102362],
                                               [-0.01102362, -0.57716533]]))

In [10]:
# THIS IS A TEST CELL!
### BEGIN HIDDEN TESTS
assert np.allclose( cournot_J([0.8395676,  0.68879643], [0.6, 0.8], 1.6), np.array([[-0.94737259, -0.0336748 ],
                                                                                    [-0.08396187, -1.19765966]]))
### END HIDDEN TESTS

In [11]:
## (c)

def my_newton(q, cournot, cournot_J, c, eta, maxit = 1000):
    """
    Implements Newton's method for a vector-valued function
    
    ((2,) np.array, fun, fun, (2,) np.array, float) -> (2,) np.array
    """    
    ### BEGIN SOLUTION
    tol = 1e-8
    tol2 = 1e-6
    
    dist = 1
    it = 0
    while dist > tol and it < maxit:
        it += 1
        f, J = cournot(q, c, eta), cournot_J(q, c, eta)
        q_new = q - np.linalg.inv(J) @ f
        dist = np.linalg.norm(q - q_new) / (1 + np.linalg.norm(q))
        q = q_new
    
#     print("Number of iterations = {}".format(it) )
    
    if np.linalg.norm(cournot(q, c, eta)) < tol2: 
        return q
    else:
        print("No solution found!")
    
    return q
    ### END SOLUTION

In [15]:
# THIS IS A TEST CELL!
### BEGIN HIDDEN TESTS
assert np.allclose(my_newton([0.2,  0.2], cournot, cournot_J, [0.6, 0.8], 1.6), np.array([0.8395676,  0.68879643]) )
### END HIDDEN TESTS

In [16]:
my_newton([2,  2], cournot, cournot_J, [0.5, 0.5], 1.5)

array([0.90064003, 0.90064003])

## Question 3 (N)

Write a function **mysecant(f, x)** in Python that implements the pseudo-code for the secant method from the lecture. Again, test it on the function $f$ and compare the result to question 1. Make sure that your algorithm does not run indefinitely by including an upper bound for the number of iterations (**maxit**).

In [25]:
def my_secant(fun, x, maxit = 100):
    """
    Implements the secant method for a univariate scalar function
    
    (fun, float) -> float
    """        
    ### BEGIN SOLUTION
    tol1 = 1e-8  
    tol2 = 1e-8
    
    dist = 1
    it = 0
    
    x_old = 0.9 * x
    f_old = fun(x_old)

    while dist > tol1 and it < maxit:
        it += 1
        x_new, f = g_secant(fun, f_old, x, x_old)
        dist = abs(x - x_new) / (1 + abs(x))
        
        ## store "old" function and x value for next iteration
        f_old = f
        x_old = x
        
        x = x_new

#     print("Number of iterations = {}".format(it) )
    
    if abs(fun(x)) < tol2: 
        return x
    else:
        print("No solution found!")
        

def g_secant(fun, f_old, x, x_old):
    """
    Implements the update rule for the secant method
    """
    f = fun(x)
    fd = (f - f_old) / (x - x_old)

    return x - f * fd**(-1), f        
        
        
### END SOLUTION    
        

In [26]:
# test function
def fun(x):
    return np.sin(4 * (x - 0.25)) + x + x**20 - 1

assert np.allclose(my_secant(fun, 0.01), 0.408293504267931)

In [27]:
# THIS IS A TEST CELL!
### BEGIN HIDDEN TESTS
assert np.allclose( my_secant(lambda x: -x**2 + 4, 0.01), 2)
assert np.allclose( my_secant(lambda x: -x**2 + 4, -0.01), -2)
### END HIDDEN TESTS

## Question 4

Consider the neoclassical growth model from the lecture. In this question, we extend it so that the production function contains *energy* $m_t$ as a third production factor in addition to capital and labor. Hence, output is given by

\begin{equation}
    y_t = f(k_t, h_{y,t}, m_t) = A k_t^\alpha m_t^\gamma h_{y,t}^{1-\alpha-\gamma}
\end{equation}

Energy is itself produced by using a part of the labor supply:

\begin{equation}
    m_t = \rho h_{m,t}
\end{equation}

which implies that one unit of labor supply creates $\rho$ units of energy.

Solve for the steady state of the planner problem numerically. Note that lifetime utility is still given by 

\begin{equation}
    u(c_t, h_t) = \frac{c^{1-\nu}}{1-\nu} - B \frac{h_t^{1+\eta}}{1+\eta}
\end{equation}

with $h_t = h_{y,t} + h_{m,t}$. 

Proceed in the following steps:

(a) Define a dictionary named **param** that contains the name of the parameters as keys and the parameter values as values. Include **alpha**, **beta**, **gamma** ($\gamma$), **delta**, **nu**, **eta**, **rho** ($\rho$), **A** and **B**. The parameter values are given below. 

(b) Define a function **cd** that takes as inputs a flat array with length 3 and a dictionary, and implements the Cobb-Douglas production function. Similarly, define a function **cd_diff** that takes the same inputs as **cd** and returns the derivatives of the Cobb-Douglas function with respect to each of the three production factors.

(c) Define a function **steady** that takes as inputs a flat array with length 3, a dictionary, as well as two functions, corresponding to **cd** and **cd_diff** defined in question (b), and implements the steady-state conditions of the model as a system of linear equations. Make sure that the function considers only *non-negative* quantities of $k$, $h_y$ and $h_m$. 

(d) Define a function **solve_steady** that takes as inputs a function (corresponding to **steady** defined in question (c)) plus the inputs that **steady** takes, and returns an array with the steady state values of $k$, $h_y$ and $h_m$ (in this order!).

In [2]:
## utility
beta = 0.9      # discount factor
nu = 2       # risk-aversion coefficient for consumption
eta = 1         # elasticity parameter for labor supply

## production
alpha = 0.25
gamma = 0.05
rho = 0.9
delta = 0.1
## derived
A = 0.844444     # productivity
B = 0.350991     # parameter for utility function

In [3]:
## (a)

### BEGIN SOLUTION
param = {'beta': beta,
         'nu': nu,
         'eta': eta,
         'delta': delta,
         'B': B,
         'alpha': alpha,
         'gamma': gamma,
         'rho': rho,
         'A': A
        }
### END SOLUTION

In [4]:
# THIS IS A TEST CELL!
### BEGIN HIDDEN TESTS
assert param['alpha'] == 0.25
assert param['delta'] == 0.1
assert param['gamma'] == 0.05
### END HIDDEN TESTS

In [5]:
## (b)
def cd(x, param):
    """
    Evaluates the Cobb-Douglas function with coefficient alpha and shift parameter A, for two inputs (x)
    
    ((3,) np.array, dict) -> float   
    """
    ### BEGIN SOLUTION
    alpha, gamma, rho, A = param['alpha'], param['gamma'], param['rho'], param['A']
    return A * x[0]**alpha * (rho * x[2])**(gamma) * x[1]**(1 - alpha - gamma)
    ### END SOLUTION

def cd_diff(x, param):
    """
    Returns the first derivative of the cobb douglas wrt k, hy and hm
    
    ((3,) np.array, dict) -> (3,) np.array
    """
    ### BEGIN SOLUTION
    alpha, gamma, A = param['alpha'], param['gamma'], param['A']   
    return (alpha * cd(x, param) / x[0], 
            (1 - alpha - gamma) * cd(x, param) / x[1],
            gamma * cd(x, param) / x[2])
    ### END SOLUTION

In [6]:
## NOTE: running this test cell assumes that you have defined a dictionary named "param" that contains the parameter values
x = np.array([2, 0.1, 0.8])
cd(x, param)
assert np.allclose(cd(x, param), 0.19710389502522407) 

In [7]:
# THIS IS A TEST CELL!
### BEGIN HIDDEN TESTS
param = {'beta': 0.9,
 'nu': 2,
 'eta': 1,
 'delta': 0.1,
 'B': 0.3509917695473248,
 'alpha': 0.25,
 'gamma': 0.05,
 'rho': 0.9,
 'A': 0.8444444444444442}

x = np.array([1, 0.5, 0.5])

assert np.allclose(cd(x, param), 0.4994714991666903)
assert np.allclose(cd_diff(x, param), np.array((0.12486787479167258, 0.6992600988333664, 0.049947149916669036)))
### END HIDDEN TESTS

In [8]:
## (c)

def steady(x, param, cd, cd_diff):
    """
    Returns the vector-valued function consisting of the steady-state conditions 
    
    ((3,) np.array, dict, fun, fun) -> (3,) np.array
    """
    ### BEGIN SOLUTION
    y = np.zeros(3)
    X = x
    mp = cd_diff(X, param)
    
    beta, delta, nu, eta, B = param['beta'], param['delta'], param['nu'], param['eta'], param['B']
    alpha, gamma, A = param['alpha'], param['gamma'], param['A']
    
    y[0] = beta * (mp[0] + 1 - delta) - 1
    y[1] = (cd(X, param) - delta * X[0])**(-nu) * mp[1] - B * (X[2] + X[1])**eta
    y[2] = mp[1] - mp[2]
    
    return y
    ### END SOLUTION

In [10]:
# THIS IS A TEST CELL!
### BEGIN HIDDEN TESTS
param = {'beta': 0.9,
 'nu': 2,
 'eta': 1,
 'delta': 0.1,
 'B': 0.3509917695473248,
 'alpha': 0.25,
 'gamma': 0.05,
 'rho': 0.9,
 'A': 0.8444444444444442}

x = np.array((1.2529757484686115, 1.5045329392063747, 0.10746663851474107))

assert np.allclose( steady(x, param, cd, cd_diff), np.zeros(3), atol = 1e-5 )

### END HIDDEN TESTS

In [13]:
## (d)

def solve_steady(steady, x0, param, cd, cd_diff):
    '''
    Solces for the steady state of the NGM
    
    ((3,)np.array, fun, dict, fun, fun) -> (3,)np.array
    '''
    ### BEGIN SOLUTION
    res_ss = scipy.optimize.root(steady, x0, args = (param, cd, cd_diff), method = 'broyden1', options = {'line_search' : None, 'jac_options': {'alpha': 1}})

    kss = res_ss.x[0]
    hyss = res_ss.x[1]
    hmss = res_ss.x[2]

    return (kss, hyss, hmss)
    ### END SOLUTION


In [15]:
# THIS IS A TEST CELL
### BEGIN HIDDEN TESTS
x0 = np.array([1.2, 1.5, 0.1])
assert np.allclose( solve_steady(steady, x0, param, cd, cd_diff), np.array((1.2529757484686115, 1.5045329392063747, 0.10746663851474107)) )
### END HIDDEN TESTS

-------------------------------------------------------------------------------------------------------------------------------

## Appendix

### Question A.1

(a) Show that the update rule for $A^{(k+1)}$ used in Broyden's method,

\begin{equation}
 A^{(k+1)} = A^{(k)} + \frac{ \left( \mathbf{f}(\mathbf{x}^{(k+1)}) - \mathbf{f}(\mathbf{x}^{(k)}) - A^{(k)} \mathbf{p}^{(k)} \right) (\mathbf{p}^{(k)})^T}{(\mathbf{p}^{(k)})^T \mathbf{p}^{(k)}}
\end{equation}

satisfies the secant condition,

\begin{equation}
 A^{(k+1)} \mathbf{p}^{(k)} = \mathbf{f}(\mathbf{x}^{(k+1)}) - \mathbf{f}(\mathbf{x}^{(k)}).
\end{equation}

 and

\begin{equation}
 A^{(k+1)} \mathbf{q} = A^{(k)} \mathbf{q}\ \ \text{for}\ \ \mathbf{q}^{T} \mathbf{p}^{(k)} = 0
\end{equation}.

(b) To prepare question (c), Show that for any vector $\mathbf{p} \in \mathbb{R}^n$, we have 

\begin{equation}
    \left| \left| \frac{\mathbf{p}\ \mathbf{p}^T}{\mathbf{p}^T \mathbf{p} } \right| \right| = 1
\end{equation}


(c) Using the result from question (b), show that

\begin{equation}
 A^{(k+1)} \in \arg \min_{A :\ A \mathbf{p}^{(k)} = \mathbf{f}(\mathbf{x}^{(k+1)}) - \mathbf{f}(\mathbf{x}^{(k)})} ||\ A - A^{(k)} ||
\end{equation}

Hint: Use the update rule in (a) to rewrite the distance $||\ A^{(k+1)}  - A^{(k)} ||$.